# First Assignment (Chapter 2)

In [1]:
# imports
import numpy as np
from importlib import reload

import unyt

from oceanpy import constants
from oceanpy.forces import coriolis

from os_classes.ocea257 import chapter1, chapter2
from os_classes import plotting as os_plotting


from bokeh import plotting
from bokeh import models

# Analytic

## 2-1 <font color='blue'>(PS1-1)</font> 

### Jupiter

Radius:  $R_J = C_J / 2 \pi$

In [3]:
circum_eq = 448600. * unyt.km # 
R_J = circum_eq / 2 / np.pi  # Radius

### Omega

$\Omega = 2 \pi / t$

In [4]:
t_J = 9.9 * unyt.hr  # 1 day, i.e. full revolution
#
omega_J = 2 * np.pi / t_J
omega_J.to('1/s')

unyt_quantity(0.0001763, '1/s')

### Centrifugal acceleration

$a = \Omega^2 r$

In [5]:
a_centrifugal = omega_J**2 * R_J
a_centrifugal.to('m/s**2')

unyt_quantity(2.21903293, 'm/s**2')

### True gravitational acceleration

#### This must be larger than measured as the Centrifugal acceleration is directed outwards.

In [6]:
g_J_measured = 26.4 * unyt.m / unyt.s**2

In [7]:
g_J_true = g_J_measured + a_centrifugal
g_J_true

unyt_quantity(28.61903293, 'm/s**2')

----

## 2-2 Shinkasen  <font color='red'>(not assigned)</font> 

In [8]:
phi = 35 * unyt.deg

### Estimate the Coriolis acceleration 

#### Assume the train motion is purely horizontal

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [9]:
u_train = 185 * unyt.km / unyt.hr

In [10]:
f = coriolis.coriolis_parameter(phi)
f.to('1/s')

unyt_quantity(8.36518044e-05, '1/s')

In [11]:
# Acceleration
dvdt = -f * u_train
dvdt.to('m/s**2')

unyt_quantity(-0.00429877, 'm/s**2')

### This is $\approx 5 \times 10^{-4}$ of the gravitational acceleration

In [12]:
3*unyt.m * 5e-4

unyt_quantity(0.0015, 'm')

### For a track 3m wide, this implies a 1mm deflection.  Probably ignorable.

----

## 2-3 Cannonball <font color='blue'>(PS1-2)</font> 

In [13]:
phi_London = (51. + 31./60)*unyt.deg
Dt = 25 * unyt.s
u_canonball = 120 * unyt.m / unyt.s

### Deflection $\Delta y$

#### Coriolis

$\frac{dv}{dt} = -f u$

$f = 2 \Omega \sin\phi$

In [14]:
f_London = coriolis.coriolis_parameter(phi_London)
f_London.to('1/s')

unyt_quantity(0.00011416, '1/s')

In [15]:
dvdt_London = -f_London * u_canonball
dvdt_London.to('m/s**2')

unyt_quantity(-0.01369967, 'm/s**2')

In [16]:
v_L_approx = dvdt_London * Dt
v_L_approx.to('m/s')

unyt_quantity(-0.34249173, 'm/s')

### Because $v \ll u$, we will ignore 2nd order effects during the $\Delta t = 25$s and take $dv/dt$ as constant.


#### Therefore, integrating and noting $v=0$ at $t=0$, we have

$\Delta y = \frac{1}{2} \frac{dv}{dt} t^2$

In [17]:
Dy = 0.5 * dvdt_London * Dt**2
Dy.to('m')

unyt_quantity(-4.28114668, 'm')

### The cannonball is deflected 4m to the right, i.e. S (assuming it is headed E)

### Other locations

In [18]:
def deflect(phi, Dt = 25 * unyt.s, u_canonball = 120 * unyt.m / unyt.s):
    # Coriolis parameter
    f = coriolis.coriolis_parameter(phi)
    # dv/dt
    dvdt = -f * u_canonball
    # Integrate
    Dy = 0.5 * dvdt * Dt**2
    # Return
    return Dy

### Murmansk

In [19]:
phi_murmansk = (68. + 52./60)*unyt.deg
Dy_murmansk = deflect(phi_murmansk)
Dy_murmansk

unyt_quantity(-5.10126301, 'm')

### Nairobi

In [20]:
phi_nairobi = -(1. + 18./60)*unyt.deg
Dy_nairobi = deflect(phi_nairobi)
Dy_nairobi

unyt_quantity(0.12407913, 'm')

### Any phi

In [21]:
phis = np.linspace(-90, 90, 1000)*unyt.deg

In [22]:
all_deflect = deflect(phis)

In [23]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Cannonball', x_axis_label='phi (deg)',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='Deflection (m)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(phis, all_deflect, legend='Cannonball', line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

----

## <font color='blue'>PS 1-3</font> -- Gust of wind

In [24]:
phi_gust = 60.*unyt.deg
u = 0.25 * unyt.m / unyt.s

### Following section 2.3 of Cushman-Roisin, the general solution for force-free motion gives:

### $u = V \sin(ft + \phi), \;\; v = V \cos(ft + \phi)$

### with $f$ the Coriolis parameter.  And integrating these velocities gives positions:

### $x = x_0 - \frac{V}{f} \cos (ft + \phi)$

### $y = y_0 + \frac{V}{f} \sin (ft + \phi)$

### Inspection of the equations reveals the interial motion has period

### $T_p = \frac{2 \pi}{f}$

### and radius $R = \frac{V}{f}$

### Computing..

In [25]:
f_gust = coriolis.coriolis_parameter(phi_gust)
f_gust.to('1/s')

unyt_quantity(0.0001263, '1/s')

In [26]:
# Period
Tp = 2 * np.pi / f_gust
Tp.to('hr')

unyt_quantity(13.81855794, 'hr')

In [27]:
# Radius
R_gust = u / f
# Diameter
D_gust = 2 * R_gust
D_gust.to('km')

unyt_quantity(5.97715738, 'km')

### How about $\phi = 5^\circ$S?

In [28]:
f_5S = coriolis.coriolis_parameter(-5.*unyt.deg)

In [29]:
# New period
Tp_5S = Tp * f_gust / np.abs(f_5S)
Tp_5S.to('hr')

unyt_quantity(137.30847612, 'hr')

In [30]:
# New Diameter
D_5S = D_gust * f_gust / np.abs(f_5S)
D_5S.to('km')

unyt_quantity(59.39218656, 'km')

### Both the period and diameter scale as $f^{-1}$. Given that $f \propto \sin\phi$ is smaller at the equator, the period and diameter are both larger at lower latitudes.

----

## <font color='blue'>PS 1-4</font> -- Tank

### Setup

In [31]:
# Tank
D_tank = 30 * unyt.cm
R_tank = D_tank / 2
H_water = 20 * unyt.cm   # Height of water before spinning
# Rotation
T_tank = 1*unyt.minute / 30. # Period
Omega_tank = 2 * np.pi / T_tank
Omega_tank.to('1/s')
# Gravity
g = 9.8 * unyt.m / unyt.s**2

### As the tank is spun up, the water relaxes to a parabolic surface that is an equipotential between Gravity and the centrifugal force.

#### Gravitaional potential in a 'constant' field:

$\Phi_g = g h$ with $h$ the vertical displacement.

#### Potential related to the centrifugal acceleration

$\Phi_c = \int \Omega^2 r dr = \frac{1}{2} \Omega^2 r^2$

#### Equating and solving for $h$:

$h = \frac{1}{2} \frac{\Omega^2}{g} r^2$

### As anticipated, the profile is a parabola

In [32]:
def h_of_r(r):
    h = 0.5 * Omega_tank**2 / g * r**2
    return h

### Evaluated at the edge, $R_{\rm tank} = 15$ cm

In [33]:
h_of_r(R_tank)

unyt_quantity(1.1329903, 'cm')

### Let's plot

In [34]:
Rvals = np.linspace(0., R_tank, 100)
hvals = h_of_r(Rvals)

In [35]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Rotating tank', x_axis_label='Distance from the center (cm)',
                    #x_axis_type="log", y_axis_type="log",
                   y_axis_label='Relative height (cm)')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(Rvals, hvals, legend='height', line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')

p.legend.location = "top_left"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### To calculate the depth at the center $r=0$, we conserve mass (or, equivalently, volume):

### $V = \pi R^2 H$  -- Volume calculated before spinning

### Integrate our new shape from $r = 0$ to $R_{\rm tank}$ with a height $H(r=0) = H_{\rm new}$ 

### and $H(r) = H_{\rm new} + h(r)$

### with  $h(r) = \frac{1}{2} \frac{\Omega^2}{g} r^2$, as above

### we have $V = \int 2 \pi r H(r) dr = \pi R_{\rm tank}^2 H_{\rm new} + \frac{\pi}{4} \frac{\Omega^2}{g} R_{\rm tank}^4$

### Solving for $H_{\rm new}$, 

### $H_{\rm new} = (V - \frac{\pi}{4} \frac{\Omega^2}{g} R_{\rm tank}^4) / \pi R_{\rm tank}^2$

In [36]:
V = np.pi * R_tank**2 * H_water
V.to('cm**3')

unyt_quantity(14137.16694115, 'cm**3')

In [37]:
Hnew = (V - (np.pi/4) * (Omega_tank**2/g) * R_tank**4) / (np.pi * R_tank**2)
Hnew.to('cm')

unyt_quantity(19.43350485, 'cm')

### The center of the water has dropped approximately 0.5cm!

----

## <font color='blue'>PS 1-5</font> -- Merry-go round

### The major inconsistency of the merry-go round for describing the Coriolis force in isolation is that the balls will experience a significant Centrifugal force.  This will generally affect the trajectories resulting in motion distince from the Coriolis force in isolation.

### Dye drops place in a rotating tank at radius $r$ will enter the water with zero angular momenum (i.e. no rotatonal motion relative to the tank).  It will therefore travel towards the center, interact with the water via viscocity?, experience the Coriolis force and deflect accordingly.

----

## 2-4 Hockey  <font color='red'>(not assigned)</font> 

In [38]:
# Dartmouth
phi_Dartmouth = (43 + 38./60) * unyt.deg
D_inertial = 26 * unyt.m

### We desire inertial motion with diameter $D_{\rm inertial} = 26$m.

### As derived above, $R = \frac{V}{f}$ with $V$ the speed of the puck and $f$ calculated for Darthmouth.

### Rearranging:

### $V = f (D/2)$

In [39]:
f_Dartmouth = coriolis.coriolis_parameter(phi_Dartmouth)
V = f * (D_inertial/2.)
V.to('cm/s')

unyt_quantity(0.10874735, 'cm/s')

### Wow, that is slow!

----

## 2-5 Stone on the Bridge <font color='blue'>(PS 1-6)</font> 

### Setup

In [40]:
phi_bridge = 35 * unyt.deg # N
h_bridge = 300 * unyt.m
g = 9.8 * unyt.m / unyt.s**2

### Direction  -- Using Cushman-Roisin coordinate systems

### In our local, rotating frame, the rotation vector for the Earth is $\bar \Omega = \Omega \cos\phi \, \hat j + \Omega \sin\phi \, \hat k$.

### The Coriolis acceleration is:  $-2 \bar \Omega \times \bar u$ 

### A stone dropped "down" has velocity $\bar u = -w \, \hat k$ 

### Taking our cross-product and being careful with the right-hand rule, there is an acceleration in the $-\hat i$ direction.   ie.  West.

### To calculate the $\hat i$ deflection, we first calculate the time in air:

### $t = \sqrt{2h/g}$

In [41]:
t_drop = np.sqrt(2 * h_bridge / g)
t_drop.to('s')

unyt_quantity(7.82460796, 's')

### Our horizontal acceleration

### $\frac{du}{dt} = 2 \Omega w \cos\phi$ 

### and gravitational acceleration gives  $w = g t$

### implying $u = \int (du/dt) dt = \Omega g \cos \phi \; t^2$

### and last we integrate $u = dx/dt$ to get the horizontal displacement

### $x = \frac{1}{3} \Omega g \cos \phi \; t^3$  (all integral constants are zero as we started at rest at $x=0$)

In [42]:
# Numerics
x = (1/3) * unyt.Omega_earth * g * np.cos(phi_bridge) * t_drop**3
x.to('cm')

unyt_quantity(9.34783674, 'cm')

### Nearly 10cm !

----

## 2-6 Raindrops  <font color='red'>(not assigned)</font> 

In [43]:
# Setup
phi_rain = 43 * unyt.deg
H = 2500 * unyt.m
g = 9.8 * unyt.m / unyt.s**2
C = 1.3 / unyt.s # drag coefficient

### Because of drag, the raindrop will reach a terminal velocity determinved by equating forces:

$C w_{\rm terminal} = g$

### Solving for this velocity:

$w_{\rm terminal} = g/C$

In [44]:
w_terminal = g/C
w_terminal

unyt_quantity(7.53846154, 'm/s')

### This is achieved in $t_{\rm terminal} \approx 1$s, which will be much shorter than the total flight time.  And we will approximate the vertical velocity as $w = w_{\rm terminal$, a constant.

### Therefore, the rain drop will fall for 

$t_{\rm total} \approx 2500 \, {\rm m} / w_{\rm terminal}$

In [45]:
t_total = H / w_terminal
t_total

unyt_quantity(331.63265306, 's')

### Similar problem 2-5 (stone on a bridge), the rain drop will be deflected to the West with Coriolis acceleration:

### $\frac{du}{dt} = 2 \Omega w_{\rm terminal} \cos\phi$ 

### By analogy, the rain will reach a terminal horizontal velocity when the Coriolis terms equals the drag:

$C u_{\rm terminal} = 2 \Omega w_{\rm terminal} \cos\phi$

### i.e. $u_{\rm terminal} = 2 \Omega w_{\rm terminal} \cos\phi / C $

In [46]:
u_terminal = 2 * unyt.Omega_earth * w_terminal * np.cos(phi_rain) / C
u_terminal.to('m/s')

unyt_quantity(0.00061852, 'm/s')

### This is acheived in $t_\rm{u, terminal} \approx u_{\rm terminal} / (2 \Omega w_{\rm terminal} \cos\phi)$

In [47]:
t_u_terminal = 1./C
t_u_terminal

unyt_quantity(0.76923077, 's')

### Therefore, the N/S velocity will also become non-zero

### $\frac{dv}{dt} = - 2 \Omega u_{\rm terminal} \sin\phi$

### It, too, will suffer a drag and equilibrate when

###  $v_{\rm terminal} = 2 \Omega u_{\rm terminal} \sin\phi / C $

In [48]:
v_terminal = 2 * unyt.Omega_earth * u_terminal * np.sin(phi_rain) / C
v_terminal.to('m/s')

unyt_quantity(4.73232799e-08, 'm/s')

### We can safely assume that this has no substinative impact on the dynamics

### To finish

$\Delta x = t_{\rm total} * u_{\rm terminal}$

$\Delta y = t_{\rm total} * v_{\rm terminal}$

In [49]:
Dx = t_total * u_terminal
Dy = t_total * v_terminal
print("The rain drop travels Dx={:0.2f} West and Dy={:0.6f} North".format(Dx, Dy))

The rain drop travels Dx=0.21 m West and Dy=0.000016 m North


----

## 2-8  Charges <font color='red'>(not assigned)</font> 

### Let $\bar r_1$ be the position of particle 1 in the rotating frame and likewise for particle 2.

### The equation of motion for particle 1 is then:

### $\frac{d^2 \bar r_1}{dt^2} = \frac{q^2 \hat r_{12}}{| \bar r_1 - \bar r_2|^2} - 2 \bar \Omega \times \frac{d \bar r_1}{dt}$ with $\hat r_{12}$ the direction from 1 to 2.

### We have a similar equation for particle 2 and we note $\hat r_{12} = - \hat r_{21}$

### And our initial conditions at $t=0$ are: $|\bar r_1 - \bar r_2| = L$ and $d\bar r_1/dt = d\bar r_2/dt = 0$

### Center of Mass:  COM = $M (\bar r_1 + \bar r_2)/2$

### We can sum the equations of motion for each particle, to recover:

### $\frac{d^2 (\bar r_1 + \bar r_2)}{dt^2} = -2 \bar\Omega \times \frac{d}{dt} (\bar r_1 + \bar r_2)$

### which we recognize as our inertial equation which has solutions of the form $u = V \sin(f t + \phi)$

### and which conserves $u^2 + v^2$.  Given our initial conditions $u^2 + v^2 = 0$, we have the COM remains at rest.

----

# Numerical

## 2-1 Accuracy

### For the semi-implicit scheme ($\alpha = 1/2$), the amplitude of oscillations are correct but the angular frequency differs from the true value, given by:

### $\tilde{f} = \frac{1}{\Delta t} \arctan \left ( \frac{f \Delta t}{1-f^2 \Delta t^2/4} \right )$

### Let $N = 2\pi/(f \Delta t)$ be the number of time steps, then $f \Delta t = 2\pi/N$ and

### $\tilde{f} = \frac{f N}{2 \pi} \arctan \left ( \frac{2\pi/N}{1-\pi^2/N^2} \right )$

In [50]:
Nvals = np.arange(100) + 2
rel_error = 1. - (Nvals/2/np.pi) * np.arctan( (2*np.pi/Nvals) / (1- np.pi**2/Nvals**2))

In [51]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Accuracy', x_axis_label='N',
                    #x_axis_type="log", 
                    y_axis_type="log",
                   y_axis_label='Relative error')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(Nvals, rel_error, line_width=2)
#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


#p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

In [52]:
good = rel_error < 0.01
Nvals[good][0]

18

----

## 2-2 Euler squared

### Euler for velocity:

### $\tilde{u}^{n+1} = \tilde{u}^n + f \Delta t \, \tilde{v}^n$

### And our positions are:

### $\tilde{x}^{n+1} = \tilde{x}^n + \Delta t \, \tilde{u}^n$

### Initial conditions


### At $t=0$, let us assume $u(t=0) = u_0, v(t=0) = 0$ and $x(t=0) = y(t=0) = 0$

In [53]:
# Init
u0 = 1.
f = 1e-2
Dt = 0.1

In [54]:
# one cycle
n = int(np.round(2 * np.pi / (f*Dt)))
n

6283

In [55]:
reload(chapter2)
x, y, u, v = chapter2.euler_inertial(n, u0, f, Dt)

In [56]:
t = np.arange(len(x)) * Dt

In [57]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Euler+Inertial Velocity', x_axis_label='t',
                    #x_axis_type="log", 
                    #y_axis_type="log",
                   y_axis_label='Velocity')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(t, u, line_width=2, legend='u')
p.line(t, v, line_width=2, legend='v', color='red')

#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


p.legend.location = "bottom_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

In [58]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Euler+Inertial Position', x_axis_label='t',
                    #x_axis_type="log", 
                    #y_axis_type="log",
                   y_axis_label='Position')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(t, x, line_width=2)#, legend='x')
p.line(t, y, line_width=2, color='red')#, legend='y', color='red')

#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


#p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

In [59]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Euler+Inertial Position', x_axis_label='x',
                    #x_axis_type="log", 
                    #y_axis_type="log",
                   y_axis_label='y')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(x,y, line_width=2)#, legend='x')
#p.line(t, y, line_width=2, color='red')#, legend='y', color='red')

#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


#p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

In [60]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Euler+Inertial KE', x_axis_label='t',
                    #x_axis_type="log", 
                    #y_axis_type="log",
                   y_axis_label='u^2 + v^2')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.line(t, u**2 + v**2, line_width=2)#, legend='x')
#p.line(t, y, line_width=2, color='red')#, legend='y', color='red')

#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


#p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### Our circle's radius is increasing in time!

----

## 2.3 Semi-implicit

### Some algebra gives:

### $\tilde{u}^{n+1} [1 + \frac{1}{4} (f \Delta t)^2] = f \Delta t \tilde{v}^n + \tilde{u}^n [1-\frac{1}{4} (f\Delta t)^2]$

### $\tilde{v}^{n+1} [1 - \frac{1}{4} (f \Delta t)^2] = -f \Delta t \tilde{u}^n + \tilde{v}^n [1+\frac{1}{4} (f\Delta t)^2]$

In [62]:
# Init
u0 = 1.
f = 1e-2
Dt = 10.
# one cycle
n_one = int(np.round(2 * np.pi / (f*Dt)))
n_one

63

In [73]:
reload(chapter2)
u, v = chapter2.semi_implicit_inertial(1000*n_one, u0, f, Dt)

In [74]:
alln = np.arange(u.size)
step = np.arange(u.size) / n_one

In [75]:
u_true = u0 * np.cos(alln*f*Dt)

In [76]:
u_true[-1] - u[-1]

-0.9930833723649835

In [79]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Semi+Inertial u', x_axis_label='Cycles',
                    #x_axis_type="log", 
                    #y_axis_type="log",
                   y_axis_label='Delta u')#, x_range=(10**-4, 1.),
                   #y_range=(1e-12, 1))

# Plot em
p.scatter(step, u-u_true, line_width=2, legend='du')
#p.line(step, u_true, line_width=2, legend='true', color='red')


#p.line(t, y, line_width=2, color='red')#, legend='y', color='red')

#p.scatter(kdx_vals, error_fb, legend='fb', line_width=2, color='red')
#p.scatter(kdx_vals, error_sc, legend='sc', line_width=2, color='green')
#p.scatter(kdx_vals, error_4, legend='4', line_width=2, color='black')


#p.legend.location = "top_right"

os_plotting.bokeh_set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### 180deg of phase at maximum difference

In [81]:
cmax = np.argmax(np.abs(u-u_true))
step[cmax]

599.3968253968254

### From equation 2.34

### $\tilde{f} = \frac{1}{\Delta t} \, \arctan \left ( \frac{f \Delta t}{1 - f^2 \Delta t^2/4} \right )$ 

In [82]:
ftilde = (1./Dt) * np.arctan( f*Dt / (1- f**2 * Dt**2/4))
f, ftilde

(0.01, 0.009991679144388553)

In [86]:
# And we find
step[cmax] - ftilde * step[cmax] / f

0.49874944380871966

### i.e. 1/2 step